In [1]:
import albumentations as A
import cv2
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [2]:
# Augmentations use paraller processing,,,

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=30, p=0.5),
    A.GaussianBlur(blur_limit=3, p=0.2),
], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))


In [3]:
def load_yolo_labels(label_path):
    if not os.path.exists(label_path):
        return []
    with open(label_path, "r") as f:
        return [list(map(float, line.strip().split())) for line in f.readlines()]

In [4]:
def save_yolo_labels(label_path, labels):
    with open(label_path, "w") as f:
        for label in labels:
            f.write(" ".join(map(str, label)) + "\n")

In [5]:
def augment_image(image_path, label_path, output_dir):
    image = cv2.imread(image_path)
    if image is None:
        return


In [6]:
def augment_image(image_path, label_path, output_dir):
    image = cv2.imread(image_path)
    if image is None:
        return

    h, w = image.shape[:2]
    labels = load_yolo_labels(label_path)
    
    if not labels:
        return

    bboxes = [[x, y, bw, bh] for _, x, y, bw, bh in labels]
    class_labels = [label[0] for label in labels]

    for i in range(3):  # 3 augmentations per image
        augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_img = augmented["image"]
        aug_bboxes = augmented["bboxes"]

        new_labels = [[cls] + list(bbox) for bbox, cls in zip(aug_bboxes, class_labels)]

        output_image_path = os.path.join(output_dir, "images", f"aug_{i}_{os.path.basename(image_path)}")
        output_label_path = os.path.join(output_dir, "labels", f"aug_{i}_{os.path.basename(label_path)}")
        
        cv2.imwrite(output_image_path, aug_img)
        save_yolo_labels(output_label_path, new_labels)


In [7]:
def process_image(image_file):
    image_path = os.path.join(r"F:\camera\accident detection\dataset\train\images", image_file)
    label_path = os.path.join(r"F:\camera\accident detection\dataset\train\labels", image_file.replace(".jpg", ".txt").replace(".png", ".txt"))
    augment_image(image_path, label_path, r"F:\camera\accident detection\delete")


In [8]:
if __name__ == "__main__":
    os.makedirs(r"F:\camera\accident detection\delete\images", exist_ok=True)
    os.makedirs(r"F:\camera\accident detection\delete\labels", exist_ok=True)

    image_files = [f for f in os.listdir(r"F:\camera\accident detection\dataset\train\images") if f.endswith((".jpg", ".png"))]

    with ThreadPoolExecutor() as executor:
        list(tqdm(executor.map(process_image, image_files), total=len(image_files)))

    print("Fast Albumentations augmentation completed! 🚀")


 85%|████████████████████████████████████████████████████████████████▊           | 10472/12272 [02:54<00:30, 59.93it/s]


OSError: [Errno 28] No space left on device